In [1]:
import pandas as pd 
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import folium
from geojson import Feature, FeatureCollection, Point
import json
from scipy.spatial import ConvexHull, convex_hull_plot_2d
from folium import IFrame
from pyproj import Proj
from shapely.geometry import Polygon

In [2]:
streetSegs = pd.read_csv("data/source data/streets.csv")
bikeData = pd.read_csv("data/modified data/bikes.csv")

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (17,18,19,20,21,22,23,25,43,71,76,77,83) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
streetSegs

,ROUTEID,FROMMEASURE,TOMEASURE,ROUTENAME,ROADTYPE,SUBBLOCKKEY,TOTALTRAVELLANES,TOTALPARKINGLANES,TOTALRAISEDBUFFERS,TOTALTRAVELLANEWIDTH,...,SMD_ID,ANC_ID,NETWORK_LOGICAL_END,AADT_COMBINATION,AADT_COMBINATION_YEAR,AADT_SINGLE_UNIT,AADT_SINGLE_UNIT_YEAR,OBJECTID,SHAPELEN,SHAPE
0,12036722,367.419586,440.347412,FLORIDA AVE NE,1,1ba98e65da8445f5af613b7d7926ee4b,5,0,0,50,...,NaN,NaN,NaN,225.0,2020.0,698.0,2020.0,2039311,0,NaN
1,12004802,787.497620,833.697327,48TH ST NE,1,574ef50353126406de8dd26c143678d7,2,2,0,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2039312,0,NaN
2,11094052,4361.677246,4474.850098,WISCONSIN AVE NW,1,41e78f2d26035a23543483aa6fe72f9b,4,2,0,40,...,NaN,NaN,NaN,502.0,2020.0,1083.0,2020.0,2039313,0,NaN
3,12001802,626.804504,667.046570,18TH ST NE,1,d586d4fa5b885eb0eca0b47d64adaf37,2,2,0,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2039314,0,NaN
4,12025352,676.623901,740.091675,CONSTITUTION AVE NE,1,3ab90e428bb82dbb283dcd83ff8985a9,2,1,0,23,...,NaN,NaN,NaN,58.0,2020.0,122.0,2020.0,2039315,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34535,11094252,852.288879,894.164978,WOODLEY RD NW,1,26abdd411387e8a811b19536bd273764,2,1,0,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2073846,0,NaN
34536,12029642,345.336700,440.631805,DIVISION AVE NE,1,e5405912fe27d34dc3b17e5353198d28,2,2,0,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2073847,0,NaN
34537,12004402,366.681610,447.191711,44TH ST NE,1,6a992fe5c9853302c5528b50106af472,2,0,0,32,...,NaN,NaN,NaN,25.0,2020.0,335.0,2020.0,2073848,0,NaN
34538,11092692,5212.461914,5309.463379,WESTERN AVE NW,1,195d6c869ebae4cd9d664cda815dd5e0,2,2,0,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2073849,0,NaN


In [4]:
bikeData

,TOTAL_BICYCLES,TOTAL_VEHICLES,TOTAL_PEDESTRIANS,LATITUDE,LONGITUDE,XCOORD,YCOORD,FATAL_BICYCLIST,MPDLATITUDE,MPDLONGITUDE,FROMDATE,STREETSEGID,ROUTEID,NEARESTINTSTREETNAME,NEARESTINTROUTEID,OFFINTERSECTION
0,1,1,0,38.895027,-76.992645,400638.075,136353.275,0,38.895027,-76.992846,2021-06-17 04:00:00+00:00,NaN,12001002,D ST NE,12026912,26.88
1,1,1,0,38.911496,-76.997027,400257.808,138181.419,0,38.911306,-76.996893,2021-06-17 04:00:00+00:00,NaN,12064672,Driveway-58012672,58012672,41.54
2,1,0,0,38.925167,-77.038769,396638.176,139699.723,0,0.000000,0.000000,2021-11-21 09:00:00+00:00,NaN,11024632,Driveway-58011442,58011442,26.25
3,1,0,0,38.884081,-77.021907,398099.244,135138.367,0,0.000000,0.000000,2021-11-27 03:29:00+00:00,NaN,14000702,D ST SW,14026912,83.99
4,1,1,0,38.917002,-77.036925,396797.709,138793.291,0,38.917086,-77.036445,2021-06-22 04:00:00+00:00,NaN,11001602,NEW HAMPSHIRE AVE NW,11064222,9.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3089,1,1,0,38.913678,-77.016350,398581.978,138423.708,0,38.914402,-77.017637,2021-06-01 04:00:00+00:00,NaN,11036722,RHODE ISLAND AVE NW,11075462,78.37
3090,1,1,0,38.898908,-76.992646,400637.950,136784.040,0,38.898908,-76.992649,2021-06-01 04:00:00+00:00,NaN,12001002,G ST NE,12038612,0.07
3091,1,1,0,38.916655,-77.008908,399227.475,138754.124,0,38.916751,-77.008870,2021-06-01 04:00:00+00:00,NaN,12075462,RHODE ISLAND AVE NW,11075462,10.96
3092,1,1,0,38.912739,-77.012552,398911.404,138319.430,0,38.912633,-77.012220,2021-06-05 04:00:00+00:00,NaN,11074162,Alley-47070502,47070502,27.60


In [59]:
BIKELANE_PARKINGLANE_ADJACENT
BIKELANE_THROUGHLANE_ADJACENT
BIKELANE_POCKETLANE_ADJACENT
BIKELANE_CONTRAFLOW
BIKELANE_CONVENTIONAL
BIKELANE_DUAL_PROTECTED
BIKELANE_DUAL_BUFFERED
BIKELANE_PROTECTED

NameError: name 'BIKELANE_PARKINGLANE_ADJACENT' is not defined

In [86]:
def runCluster(dataset, eps, minSamples):
    accPoints = np.empty((0, 2))   
    
    for lat,long in zip(dataset['LATITUDE'], dataset['LONGITUDE']):
        accPoints = np.append(accPoints, np.array([[lat,long]]), axis=0)

    # Compute DBSCAN
    db = DBSCAN(eps=eps, min_samples=minSamples).fit(accPoints)

    clusters = db.fit_predict(accPoints)
    dataset['CLUSTER'] = clusters
    

    clusters = dataset['CLUSTER'].unique()   
    clusterDict = {}
    primaryStreetDict = {}
    secondaryStreetDict = {}
    primaryFacilityTypeDict = {}
    secondaryFacilityTypeDict = {}
    primaryStreetSegIdDict = {}
    secondaryStreetSegIdDict = {}
    for i in clusters:
        clusterSet = dataset[(dataset["CLUSTER"] == i)]
        valueCounts = clusterSet['ROUTEID'].value_counts()
        if (len(valueCounts.index) > 1 and valueCounts.index[1] != "Route not found"):
            primaryValueId = valueCounts.index[0]
            primaryValue = streetSegs[streetSegs["ROUTEID"] == primaryValueId]["ROUTENAME"].value_counts().index[0]
            secondaryValueId = valueCounts.index[1]
            secondaryValue = streetSegs[streetSegs["ROUTEID"] == secondaryValueId]["ROUTENAME"].value_counts().index[0]
            primaryStreetDict[i] = primaryValue
            secondaryStreetDict[i] = secondaryValue
        elif (len(valueCounts.index) > 0):
            primaryValueId = valueCounts.index[0]
            primaryValue = streetSegs[streetSegs["ROUTEID"] == primaryValueId]["ROUTENAME"].value_counts().index[0]
            primaryStreetDict[i] = primaryValue
            secondaryStreetDict[i] = "None"
        else:
            primaryStreetDict[i] = "None"
            secondaryStreetDict[i] = "None"

    primaryStreets = []
    secondaryStreets = []

    for i in dataset["CLUSTER"]:
        if (primaryStreetDict[i] is "None"):
            primaryStreets.append("None")
            primaryStreetSegIds.append("None")
        else:
            primaryStreets.append(primaryStreetDict[i])
        if (secondaryStreetDict[i] is "None"):
            secondaryStreets.append("None")
        else:
            secondaryStreets.append(secondaryStreetDict[i])
       
    dataset['CLUSTERPRIMARYSTREET'] = primaryStreets
    dataset['CLUSTERSECONDARYSTREET'] = secondaryStreets
    return dataset

<>:45: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:50: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:45: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:50: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-86-4670fc389b3b>:45: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (primaryStreetDict[i] is "None"):
<ipython-input-86-4670fc389b3b>:50: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (secondaryStreetDict[i] is "None"):


In [112]:
colorsList = ['red', 'blue', 'black', 'purple', 'orange', 'pink', 'green']

def getColor(ind):
    colorLen = len(colorsList)
    colorNum = ind%colorLen
    color = colorsList[colorNum]
    return color

def makeHulls(dataset):
    features = []
    clusterDict = {}
    primaryStreetDict = {}
    secondaryStreetDict = {}
#     primaryFacilityTypeDict = {}
#     secondaryFacilityTypeDict = {}
#     primaryStreetSegIdDict = {}
#     secondaryStreetSegIdDict = {}
    totalAccidentNumberDict = {}
    #for lat, long, cluster, primaryStreet, secondaryStreet, primaryFaciltyType, secondaryFaciltyType, primaryStreetSegId, secondaryStreetSegId in zip(dataset['LATITUDE'],dataset['LONGITUDE'],dataset['CLUSTER'],dataset['CLUSTERPRIMARYSTREET'],dataset['CLUSTERSECONDARYSTREET'],dataset['CLUSTERPRIMARYFACILITYTYPE'],dataset['CLUSTERSECONDARYFACILITYTYPE'], dataset['CLUSTERPRIMARYSTREETSEGID'], dataset['CLUSTERSECONDARYSTREETSEGID']):
    for lat, long, cluster, primaryStreet, secondaryStreet in zip(dataset['LATITUDE'],dataset['LONGITUDE'],dataset['CLUSTER'],dataset['CLUSTERPRIMARYSTREET'],dataset['CLUSTERSECONDARYSTREET']):
        if (cluster != -1):
            if (cluster in clusterDict.keys()):
                thisCluster = clusterDict[cluster]
                thisCluster.append([long,lat])
                clusterDict[cluster] = thisCluster
                totalAccidentNumberDict[cluster] +=1
            else:
                clusterDict[cluster] = [[long,lat]]
                primaryStreetDict[cluster] = str(primaryStreet)
                secondaryStreetDict[cluster] = str(secondaryStreet)
#                 primaryFacilityTypeDict[cluster] = str(primaryFaciltyType)
#                 secondaryFacilityTypeDict[cluster] = str(secondaryFaciltyType)
#                 primaryStreetSegIdDict[cluster] = str(primaryStreetSegId)
#                 secondaryStreetSegIdDict[cluster] = str(secondaryStreetSegId)
                totalAccidentNumberDict[cluster] = 0

    hulls = []
    clusters = []
    primaryStreets = []
    secondaryStreets = []
#     primaryFacilityTypes = []
#     secondaryFacilityTypes = []
    primaryStreetSegIds = []
    secondaryStreetSegIds = []
    totalAccidentNumbers = []
    clusterAreas = []
    for cluster in clusterDict.keys():
        clusters.append(cluster)
        thisHull = ConvexHull(clusterDict[cluster])
        hulls.append(thisHull)
        primaryStreets.append(primaryStreetDict[cluster])
        secondaryStreets.append(secondaryStreetDict[cluster])
#         primaryFacilityTypes.append(primaryFacilityTypeDict[cluster])
#         secondaryFacilityTypes.append(secondaryFacilityTypeDict[cluster])
#         primaryStreetSegIds.append(primaryStreetSegIdDict[cluster])
#         secondaryStreetSegIds.append(secondaryStreetSegIdDict[cluster])
        totalAccidentNumbers.append(totalAccidentNumberDict[cluster])
        clusterAreas.append(area(thisHull))
        
    clusters = pd.DataFrame(
            {
                "hull" : hulls,
                "cluster" : clusters,
                "primaryStreet" : primaryStreets,
                "secondaryStreet" : secondaryStreets,
                #"primaryFacilityType" : primaryFacilityTypes, 
                #"secondaryFacilityType" : secondaryFacilityTypes,
#                 "primaryStreetSegId" : primaryStreetSegIds,
#                 "secondaryStreetSegId" : secondaryStreetSegIds,
                "totalAccidents" : totalAccidentNumbers,
                "clusterArea" : clusterAreas
            })
        
    return clusters


def makeGeoJson(clusters):
    shapes = []

    for cluster, hull in zip(clusters['cluster'], clusters['hull']):
        outline = []
        for p in hull.vertices:
            outline.append(list(hull.points[p]))
        outline.append(list(hull.points[hull.vertices[0]]))
        shapes.append(outline)

    clusters["shape"] = shapes
    
    myGeoJson = {"type": "FeatureCollection","features":[]}   
    
    for index, row in clusters.iterrows():
        shape = row['shape']
        primaryStreet = row['primaryStreet']
        secondaryStreet = row['secondaryStreet']
#         primaryFacilityType = row['primaryFacilityType']
#         secondaryFaciltyType = row['secondaryFacilityType']
#         primaryStreetSegId = row['primaryStreetSegId']
#         secondaryStreetSegId = row['secondaryStreetSegId']
        totalAccidents = row['totalAccidents']
        clusterArea = row['clusterArea']
        thisFeature = {
            "type": "Feature",
            "properties": {
                "name": str(shape),
                "primaryStreet" : primaryStreet,
                "secondaryStreet" : secondaryStreet,
#                 "primaryFacilityType" : primaryFacilityType,
#                 "secondaryFaciltyType" : secondaryFaciltyType,
#                 "primaryStreetSegId" : primaryStreetSegId,
#                 "secondaryStreetSegId" : secondaryStreetSegId,
                "totalAccidents" : totalAccidents,
                "clusterArea" : clusterArea
             },
            "geometry" : {
                "type" : "Polygon",
                "coordinates" : [shape]
            }
        }
        myGeoJson['features'].append(thisFeature)

    geo_str = json.dumps(myGeoJson)
    return geo_str, clusters

def area(hull):
    verts = [hull._points[i] for i in hull._vertices]
    polygon = Polygon(verts)
    polyArea = 1000000 * polygon.area
    return polyArea


def style_function(feature):
#     print(feature['properties'])
    # styling guide  https://python-visualization.github.io/folium/modules.html
    fillColor = ""
#     if (feature['properties']['primaryFacilityType'] == "None"):
    return {
        'fillColor': 'red',
        'fillOpacity' : .5,
        'color' : 'black'
    }
#     elif (feature['properties']['primaryFacilityType'] == "Shared Lane"):
#         return {
#             'fillColor': 'orange',
#             'fillOpacity' : .5,
#             'color' : 'black'
#         }
#     elif (feature['properties']['primaryFacilityType'] == "Contraflow Bike Lane"):
#         return {
#             'fillColor': 'purple',
#             'fillOpacity' : .5,
#             'color' : 'black'
#         }
#     elif (feature['properties']['primaryFacilityType'] == "Existing Bike Lane"):
#         return {
#             'fillColor': 'blue',
#             'fillOpacity' : .5,
#             'color' : 'black'
#         }
#     elif (feature['properties']['primaryFacilityType'] == "Climbing Lane"):
#         return {
#             'fillColor': 'pink',
#             'fillOpacity' : .5,
#             'color' : 'black'
#         }
#     elif (feature['properties']['primaryFacilityType'] == "Cycle Track"):
#         return {
#             'fillColor': 'green',
#             'fillOpacity' : .5,
#             'color' : 'black'
#         }
#     else:
#         return {
#             'fillColor': 'black',
#             'fillOpacity' : .5,
#             'color' : 'black'
#         }

def mapAll(dataset):

    dcMap = folium.Map(location=[38.9072, -77.0369], zoom_start=13)

    featureGroupDict = {}
    featureGroupDict[1] = folium.FeatureGroup(name="bike fatalities")
    featureGroupDict[2] = folium.FeatureGroup(name="bike incident")
    counter = 0
    for lat, long, fb in zip(dataset['LATITUDE'], dataset['LONGITUDE'], dataset['FATAL_BICYCLIST']):
        if (fb > 0):
            folium.CircleMarker(location=[lat, long], radius = 2, color = 'red').add_to(featureGroupDict[1])
        else:
            folium.CircleMarker(location=[lat, long], radius = .5, color = 'black').add_to(featureGroupDict[2])
        
    for key in featureGroupDict.keys():
        dcMap.add_child(featureGroupDict[key])

    folium.map.LayerControl('topright', collapsed=False).add_to(dcMap)
    return dcMap

def makeMap(dataset):
    # plotting all bike accidents
    dcMap = folium.Map(location=[38.9072, -77.0369], zoom_start=13)

    featureGroupDict = {}
        
    counter = 0
    for lat, long, cluster, clusterprimaryStreet, clusterSecondaryStreet, fb in zip(dataset['LATITUDE'], dataset['LONGITUDE'], dataset["CLUSTER"], dataset["CLUSTERPRIMARYSTREET"], dataset["CLUSTERSECONDARYSTREET"], dataset['FATAL_BICYCLIST']):
        if (cluster != -1):
            if (cluster not in featureGroupDict.keys()):
                featureGroupDict[cluster] = folium.FeatureGroup(name=(str(clusterprimaryStreet) + " and " + str(clusterSecondaryStreet)))
            if (fb > 0):
                folium.CircleMarker(location=[lat, long], radius = 2, color = 'red').add_to(featureGroupDict[cluster])
            else:
                folium.CircleMarker(location=[lat, long], radius = .5, color = 'black').add_to(featureGroupDict[cluster])

        counter = counter + 1 
        
    clusters = makeHulls(dataset)
    geoString, clusters = makeGeoJson(clusters)
    
    text_file = open("data/cluster data/geoJSONs/" + str(len(clusters)) + "geoJSON.geojson", "w")
    text_file.write(geoString)
    text_file.close()
                     
    featureGroupDict["Layers"] = folium.FeatureGroup(name="Layers")
#     ourLayers = folium.GeoJson(json.loads(geoString), style_function=style_function,
#                                tooltip=folium.features.GeoJsonTooltip(
#                                 fields=['primaryStreet','secondaryStreet','primaryFacilityType','secondaryFaciltyType','primaryStreetSegId','secondaryStreetSegId']
#                               ))
    ourLayers = folium.GeoJson(json.loads(geoString), style_function=style_function,
                               tooltip=folium.features.GeoJsonTooltip(
                                fields=['primaryStreet','secondaryStreet']
                              ))
    ourLayers.add_to(featureGroupDict["Layers"]);
    
    for key in featureGroupDict.keys():
        dcMap.add_child(featureGroupDict[key])
        
    folium.map.LayerControl('topright', collapsed=False).add_to(dcMap)

    item_txt = """<br> &nbsp; {item} &nbsp; <i class="fa fa-map-marker fa-2x" style="color:{col}"></i>"""
    html_itms = item_txt.format(item= "None" , col= "red")

    legend_html = """
     <div style="
     position: fixed; 
     bottom: 5px; left: 5px; width: 220px; height: 160px; 
     border:2px solid grey; z-index:9999; 
     background-color:white;
     opacity: .85;
     font-size:14px;
     font-weight: bold;
     ">
     <div style= "text-align:center">Street Bike Facilities</div>
     <br>
     <!--<div><div style="margin-left:20px;">None </div><div style="margin-top:-14px;margin-left:180px;height:10px;width:10px;background-color:red;">  </div></div>
     <div><div style="margin-left:20px;">Shared Lane </div><div style="margin-top:-14px;margin-left:180px;height:10px;width:10px;background-color:orange;">  </div></div>
     <div><div style="margin-left:20px;">Contraflow Bike Lane </div><div style="margin-top:-14px;margin-left:180px;height:10px;width:10px;background-color:purple;">  </div></div>
     <div><div style="margin-left:20px;">Existing Lane </div><div style="margin-top:-14px;margin-left:180px;height:10px;width:10px;background-color:blue;">  </div></div>
     <div><div style="margin-left:20px;">Climbing Lane </div><div style="margin-top:-14px;margin-left:180px;height:10px;width:10px;background-color:pink;">  </div></div>
     <div><div style="margin-left:20px;">Cycle Track </div><div style="margin-top:-14px;margin-left:180px;height:10px;width:10px;background-color:green;">  </div></div>
     -->
     <div><div style="margin-left:20px;">Some Bike Facility </div><div style="margin-top:-14px;margin-left:180px;height:10px;width:10px;background-color:red;">  </div></div>  
      
      </div> """
    dcMap.get_root().html.add_child(folium.Element( legend_html ))
    
    framedData = pd.read_json(geoString)
#     clusters.to_csv("data/cluster data/" + str(len(framedData)) + "clustersalldata.csv")
    
    return dcMap

In [113]:
clusteredDataSmall = runCluster(bikeData, 0.0015, 25)
dcMap = makeMap(clusteredDataSmall)
dcMap

In [114]:
clusteredDataMedium = runCluster(bikeData, 0.0014, 15)
dcMap = makeMap(clusteredDataMedium)
dcMap

In [115]:
clusteredDataLarge = runCluster(bikeData, 0.0005, 7)
dcMap = makeMap(clusteredDataLarge)
dcMap